In [1]:
%autosave 300

Autosaving every 300 seconds


In [2]:
import nltk

자연어에는 광범위한 문법 구조가 있으므로, 8장에서 다룬 간단한 방법으로 다루기 어렵다. 유연성을 높이기 위해 S, NP 그리고 V와 같은 문법 요소들의 처리 방법을 달리 할 것이다.

이 장의 목적은 다음 질문에 답하는 것이다.

1. context-free 문법 프레임워크를 어떻게 확장하여 문법적 분류와 생산을 정교화할 수 있는가?
2. 특징 구조(feature structure)의 주요 속성은 무엇이며, 어떻게 computationally 사용할 수 있는가?
3. 특징 구조 문법을 통해 이제 어떤 종류의 언어학적 패턴과 문법 구조를 파악할 수 있는가? 

# 1. Grammatical Features

6장에서는 텍스트의 특징을 감지하는 것에 의존하는 classifier를 만드는 법을 설명했는데, 이 특징들은 단어의 마지막 글자를 추출하는 것처럼 간단할 수도 있고, 혹은 classifier로부터 예측된 POS처럼 다소 복잡할 수도 있다. 여기서는 rule-based 문법을 구현하는데 있어 특징(feature)들의 역할에 대해 공부해 본다.

In [3]:
kim = { 'CAT' : 'NP' , 'ORTH' : 'Kim' , 'REF' : 'k' }
chase = { 'CAT' : 'V' , 'ORTH' : 'chased' , 'REL' : 'chase' } 

Feature Structures(feature와 value pair)
 - 'CAT' : 문법적 범주
 - 'ORTH' : 표기법(철자법)
 - 'REF'/'REL' : 추가 정보(REFerent, RELation, etc.)

위와 같이 feature structure는 문법 개체에 대한 다양한 정보를 포함하고 있다.

그 정보가 모든 정보를 포함할 필요는 없으며, 아래와 같이 속성을 추가할 수도 있다.

In [4]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

In [5]:
sent = "Kim chased Lee"
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

"Kim chased Lee" 라는 문장에서 "chase" 앞뒤 단어를 동사의 Agent(sbj, 주체)와 Patient(obj, 객체)로 엮어서 처리하고 싶다. 이 예제에서는 동사의 왼쪽과 오른쪽에 있는 NP가 각각 주체와 객체라는 단순한 가정을 한다.

In [6]:
def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs

표기법이 맞아떨어지면 해당 토큰(단어)에 feature structure를 갖다 붙이는 함수

In [7]:
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])

In [8]:
verb

{'AGT': 'sbj', 'CAT': 'V', 'ORTH': 'chased', 'PAT': 'obj', 'REL': 'chase'}

예문에서는 agent가 k, patient가 l이므로 verb의 feature structure의 속성 변경

In [9]:
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']

In [10]:
verb

{'AGT': 'k', 'CAT': 'V', 'ORTH': 'chased', 'PAT': 'l', 'REL': 'chase'}

In [11]:
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print("%-5s => %s" % (k, verb[k]))

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


feature structure는 꽤 강력하지만 위의 예제는 사실 설명을 위해 조작한 것이다. 다음은 context-free 문법 및 구문 분석의 프레임워크가 feature structure를 도입하여 좀 더 일반적인 방식으로 구현하는 방법을 소개할 것이다.

## 1.1. Syntactic Agreement

여기서는 간단하게 문법에 대한 합의를 하고난 후 다음 장으로 넘어간다.

수 일치
 - 단수, 복수
 - 인칭

![](images/1.1.png)

각각의 문장을 3인칭 단수, 3인칭 복수 등과 같이 encoding 후 context-free grammar(CFG)를 적용한 모습이다.

![](images/missing.png)

위 (7)에서 우리는 'this dog runs'라는 문장을 생성했다. 그러나 우리는 'these dogs run'과 같은 문법적으로 올바른 다른 문장도 얻고 싶다. 가장 직관적인 방법은 grammar에 새로운 non-terminals와 productions를 추가하는 것이다.

![](images/6.png)

위 (7)에 비해 (6)은 단수형, 복수형을 구분해서 갖고있으므로 2배의 정보를 갖고있는 셈이다. 작은 문법에 있어서는 큰 문제는 안되겠지만, 영어 문법의 전반을 다루기 위해서는 그리 아름다운 size는 아니다. 더 좋은 방법은 없을까?

## 1.2. Using Attributes and Constraints

앞서 속성(properties)이라는 것을 갖는 언어학적 범주에 대해 비공식적으로 얘기했다. 예를 들어 명사는 복수형이라는 속성이 있다.

![](images/8.png)

여기서 NUM은 number의 약자로, 수 일치에 활용되는 속성이며, sg(singlular)와 pl(plural)의 값을 가질 수 있다.

__S -> NP[NUM=?n] VP[NUM=?n]__

S(sentence)는 NP(noun phrase)와 VP(verb phrase)로 구성되어 있는데 NUM이 sg 혹은 pl이든 구조는 변함이 없다. 다시 NP도 아래와 같이 나뉜다.

__NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]__

아래 (12)의 경우에는 수 일치가 잘 된 모습, (13)의 경우 수 일치에 실패(FAIL)한 모습. 
depth 1인 dog를 시작으로 앞의 this와 수 일치가 되었는지 확인하고 NP의 NUM 속성 결정

![](images/12.png)

짜잔 ~ 복수형으로 수일치가 된 문장의 tree 구조

![](images/14.png)

this와 these 같은 determiners는 단수냐 복수냐에 대한 정보가 필요하지만, the, some, any 같은 경우에는 명사의 수에 따라 선택하는 것이 아니다.

__Det[NUM=?n] -> 'the' | 'some' | 'any'__

In [12]:
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


NUM 뿐만 아니라 둘 이상의 속성도 가질 수 있다. 예를 들어, 동사의 경우에는 TENSE도 함께 들어가 시제를 나타낸다.

In [13]:
tokens = 'Kim likes children'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat0.fcfg', trace=2)

In [14]:
for tree in cp.parse(tokens):
    print(tree)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: 'sg', ?t: 'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'pl'}
Feature Single Edge Fundame

## 1.3. Terminology

지금까지는 sg, pl과 같은 feature 값들만 봐왔는데, 이런 단순한 값들을 __atomic__ 이라고 일반적으로 부른다. 즉, 더이상 서브파트로 나뉠 수 없음을 의미한다. atomic의 특수 케이스로 __boolean__ 값이 있는데, 예를 들어 __auxiliary__ 동사(조동사)인지의 여부를 나타내는 feature로 AUX를 사용한다. 

__V[TENSE=pres, AUX=+] -> 'can'__

지금까지는 문법 범주(verb, noun, etc.)에 feature annotation을 하는 경우만 봐왔지만, 아예 문법 범주 조차도 POS feature에 추가하여 표기할 수도 있다.(중요하지는 않은듯)

AGR : Agreement Features

![](images/1.3.png)

# 2. Processing Feature Structures

이번 섹션에서는 NLTK에서 feature structure가 어떻게 구성되고 조정되는지에 대해 알아볼 것이다. 또한, 합성의 방법에 대해서도 논의 하여 서로 다른 두 가지 feature structure에 포함 된 정보를 결합할 수 있게 할 것이다.

In [15]:
fs1 = nltk.FeatStruct(TENSE='past', NUM='sg')
print(fs1)

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]


In [16]:
fs1 = nltk.FeatStruct(PER=3, NUM='pl', GND='fem')
print(fs1['GND'])

fem


In [17]:
fs1['CASE'] = 'acc'

In [18]:
fs1

[CASE='acc', GND='fem', NUM='pl', PER=3]

agreement feature인 fs1을 포함하는 feature structure, fs2를 만들어 보자

In [19]:
fs2 = nltk.FeatStruct(POS='N', AGR=fs1)
print(fs2)

[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fem' ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]


In [20]:
print(fs2['AGR'])

[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]


아예 feature-value 쌍의 string의 형태로 feature structure를 선언할 수도 있다. 누가 쓸 지는 모르겠지만.

In [21]:
print(nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]"))

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


feature structure가 근본적으로는 언어학적 목적으로만 사용되는 건 아니고, __그래프__라고 여기는 것이 종종 편리하다. 특히, __directed acyclic graphs(DAGS)__와 상당히 닮았다.

__자연어 처리와는 무관한 설명이므로 거침없이 생략한다.__

## 2.1. Subsumption and Unification

feature structure는 어떤 특정 객체에 대해 __부분 정보(partial information)__를 제공하는 것으로 여기는 것이 일반적이다. 얼마나 많은 정보를 포함하는 feature structure냐에 따라 순서를 정할 수 있다.

![](images/23.png)

(23)의 경우 c가 가장 많은 정보를 포함하고 있으며, b의 경우에는 모든 정보가 c에도 포함되어 있으므로 c의 subsumption이라 할 수 있다.

어느 하나에 포함되는 feature structure의 관계가 아니라 필요한 정보를 덧붙이는 경우에는 어떨까? 주소에는 street 이름과 number 뿐만 아니라 city에 대한 정보가 필요한데, 이때 a와 b를 merge하여 c를 얻는다.

![](images/25.png)

In [22]:
fs1 = nltk.FeatStruct(NUMBER=74, STREET='rue Pascal')
fs2 = nltk.FeatStruct(CITY='Paris')

In [23]:
print(fs1.unify(fs2))

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


fs0와 fs1 모두 같은 A라는 path를 갖지만 값이 다른 경우, unification 결과는 None이 된다.

In [24]:
fs0 = nltk.FeatStruct(A='a')
fs1 = nltk.FeatStruct(A='b')
fs2 = fs0.unify(fs1)

In [25]:
print(fs2)

None


In [26]:
fs0 = nltk.FeatStruct("""[NAME=Lee, 
    ADDRESS=[NUMBER=74, STREET='rue Pascal'], 
    SPOUSE= [NAME=Kim, ADDRESS=[NUMBER=74,STREET='rue Pascal']]]""")

In [27]:
print(fs0)

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


배우자의 주소에 Paris를 CITY의 value로 추가하고 싶다. 내 부인은 빠리지엥이니까.

In [28]:
fs1 = nltk.FeatStruct("[SPOUSE = [ADDRESS = [CITY = Paris]]]")

In [29]:
print(fs1.unify(fs0))

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [           [ CITY   = 'Paris'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


structure를 share하는 structure-sharing 버전의 경우에는 조금 다른 결과를 낸다.

In [30]:
fs2 = nltk.FeatStruct("""[NAME=Lee, ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
    SPOUSE=[NAME=Kim, ADDRESS->(1)]]""")

In [31]:
print(fs2)

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


In [32]:
print(fs1.unify(fs2))

[               [ CITY   = 'Paris'      ] ]
[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


# 3. Extending a Feature based Grammar

## 3.1. Subcategorization

Q. 아래 표현식의 의미는?

 - __VP -> IV__

 - __VP -> TV NP__

자동사 타동사가 동사의 서로 다른 두 종류라는 것은 알고 있지만, 이러한 notation이 동사의 전반적인 특징을 설명해주지는 않는다. e.g. "V의 범주에 포함되는 모든 단어는 시제를 나타낼 수 있다"는 말을 할 수 없는데, 예를 들어, _walk_는 V가 아니라 IV의 범주에 들어가기 때문이다. 그렇다면 어떻게 V notation을 버리지 않고 IV와 TV를 모두 포함할 수 있을까?

![](images/28.png)

![](images/29.png)

- SBar : 아마도 that절을 의미하는 듯 -> Comp S
- Comp -> 'that'

__Q. Put the book on the table__

![](images/33.png)

## 3.2. Heads Revisited

앞서 subcategorization를 통해 동사의 '특성'을 좀 더 일반화할 수 있었다. 또 다른 종류의 특성은 다음과 같다: V는 VP의 head, N은 NP의 head, 그리고 A는 AP의 head인 것처럼 _명사절은 명사가 이끄는 절, 동사절은 동사가 이끄는 절과 같은 특성_이다. 그렇다고 모든 절이 head가 있는 것은 아니지만 문법적으로 부모/자식의 관계를 밝혀서 표기하고 싶다. 지금은 V와 VP는 단지 atomic symbol에 불과하지만, 이 두 symbol을 feature를 이용해 연관짓는 방법을 찾아낼 것이다. 우리는 방법을 찾을 것이다. 늘 그래왔듯.

[참고 블로그 포스팅](http://blog.naver.com/frost9001/220665950655)

__X-bar Syntax__(영어학 관련 이론으로, PS Rule의 오류를 보완하기 위해 Chomsky가 채택하였다고 함)에서는 이 이슈에 대해 __phrasal level(절 수준)__의 표기를 빼는 방향으로 접근한다. 

예를 들어, N이 어휘 수준이라면 N'는 한 수준 위(전통적으로는 Nom으로 표기)를, N''는 phrasal level(전통적으로는 NP로 표기)에 해당한다.

![](images/34.png)

(34)에서 N은 structure의 head이고, N'과 N''는 __N의 (phrasal) projections__이라고 부른다.

X-bar syntax의 주요 주장은 모든 구성요소가 구조적 유사성을 공유한다는 것이다. X 대신 N, V, A, P를 대입하여 사용하면 head인 X의 직속하위 범주에 속하는 _complements(보어)_는 항상 head의 sibling의 관계에 위치한다. 반면 _adjuncts(부사)_는 중간 범주(intermediate category)인 X'의 sibling의 관계에 위치한다. 

(35)에서의 두 개의 부사, P''와 (34a)의 한 개의 보어, P''는 대조적임을 확인할 수 있다.

![](images/35.png)

## 3.3. Auxiliary Verbs and Inversion

아래 두 경우는 동사와 주어의 위치가 뒤바뀐 절을 포함한 예문이다.

(37)		
 - Do you like children?
 - Can Jody walk?


(38)		
 - Rarely do you see Kim.
 - Never have I seen this dog.



(37)에서와 같이 도치구문의 맨 처음에 올 수 있는 동사를 __auxiliaries__라고 한다. 

 - __S[+INV] -> V[+AUX] NP VP__

![](images/42.png)

(43)		
 - You like Jody.
 - *You like.



(43)에서 알 수 있듯 like는 NP complement를 필요로 한다. 그러나 아래 (45)의 경우에는 like 뒤에 complement는 생략 가능하다.

(45)		
 - Kim knows who you like.
 - This music, you really like.

즉, 적당한 __filler__ 문장에 있을 경우, 예를 들어 의문사 _who_, like의 complement가 생략 가능하다. 여기서 (45)의 문장의 경우 complement가 생략된 자리에 __gap__을 포함한다고 말하는 것이 일반적이고, underscore를 통해 표기한다.

(47)
 - Which card do you put __ into the slot?
 - Which slot do you put the card into __?



즉, gap은 filler에 의해 자격을 인정받았을 때 생길 수 있다. 역으로, filler 문장 어딘가에 적절한 gap이 있을 때 생길 수 있다. 즉, 서로 필요한 존재라는 의미인듯. 오 맞음. __dependency__라고 부르기도 한다. 그러나 filler와 gap 사이의 거리가 아주 멀어도 상관 없으므로 찾기가 어려울수도 있겠다.

(50)		
 - Who do you like __?
 - Who do you claim that you like __?
 - Who do you claim that Jody says that you like __?

이러한 특징들을 총망라하여 __unbounded dependency construction__이라는 이름으로 불리게 된다; that is, a filler-gap dependency where there is no upper bound on the distance between filler and gap.

이런 문장들의 문제점이 있는데! 생략된 sub-constituent를 어디다 표기할 것인가? __slash categories__를 이용하여 Y/XP로 나타내면, XP 범주의 sub-constituent가 없는 Y 범주의 phrase라고 해석하면 된다. 예를 들어, S/NP는 NP가 없는 S라고 해석한다.

![](images/51.png)

In [33]:
nltk.data.show_cfg('grammars/book_grammars/feat1.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
S[-INV] -> NP VP
S[-INV]/?x -> NP VP/?x
S[-INV] -> NP S/NP
S[-INV] -> Adv[+NEG] S[+INV]
S[+INV] -> V[+AUX] NP VP
S[+INV]/?x -> V[+AUX] NP VP/?x
SBar -> Comp S[-INV]
SBar/?x -> Comp S[-INV]/?x
VP -> V[SUBCAT=intrans, -AUX]
VP -> V[SUBCAT=trans, -AUX] NP
VP/?x -> V[SUBCAT=trans, -AUX] NP/?x
VP -> V[SUBCAT=clause, -AUX] SBar
VP/?x -> V[SUBCAT=clause, -AUX] SBar/?x
VP -> V[+AUX] VP
VP/?x -> V[+AUX] VP/?x
# ###################
# Lexical Productions
# ###################
V[SUBCAT=intrans, -AUX] -> 'walk' | 'sing'
V[SUBCAT=trans, -AUX] -> 'see' | 'like'
V[SUBCAT=clause, -AUX] -> 'say' | 'claim'
V[+AUX] -> 'do' | 'can'
NP[-WH] -> 'you' | 'cats'
NP[+WH] -> 'who'
Adv[+NEG] -> 'rarely' | 'never'
NP/NP ->
Comp -> 'that'


In [34]:
tokens = 'who do you claim that you like'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat1.fcfg')
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[+WH] who)
  (S[+INV]/NP[]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[]/NP[]
      (V[-AUX, SUBCAT='clause'] claim)
      (SBar[]/NP[]
        (Comp[] that)
        (S[-INV]/NP[]
          (NP[-WH] you)
          (VP[]/NP[] (V[-AUX, SUBCAT='trans'] like) (NP[]/NP[] )))))))


![](images/52.png)

In [35]:
tokens = 'you claim that you like cats'.split()
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[-WH] you)
  (VP[]
    (V[-AUX, SUBCAT='clause'] claim)
    (SBar[]
      (Comp[] that)
      (S[-INV]
        (NP[-WH] you)
        (VP[] (V[-AUX, SUBCAT='trans'] like) (NP[-WH] cats))))))


In [36]:
tokens = 'rarely do you sing'.split()
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (Adv[+NEG] rarely)
  (S[+INV]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[] (V[-AUX, SUBCAT='intrans'] sing))))


## 3.5. Case and Gender in German(생략)

# 4. Summary

 - The traditional categories of context-free grammar are atomic symbols. An important motivation for feature structures is to capture fine-grained distinctions that would otherwise require a massive multiplication of atomic categories.
 - By using variables over feature values, we can express constraints in grammar productions that allow the realization of different feature specifications to be inter-dependent.
 - Typically we specify fixed values of features at the lexical level and constrain the values of features in phrases to unify with the corresponding values in their children.
 - Feature values are either atomic or complex. A particular sub-case of atomic value is the Boolean value, represented by convention as [+/- f].
 - Two features can share a value (either atomic or complex). Structures with shared values are said to be re-entrant. Shared values are represented by numerical indexes (or tags) in AVMs.
 - A path in a feature structure is a tuple of features corresponding to the labels on a sequence of arcs from the root of the graph representation.
 - Two paths are equivalent if they share a value.
 - Feature structures are partially ordered by subsumption. FS0 subsumes FS1 when all the information contained in FS0 is also present in FS1.
 - The unification of two structures FS0 and FS1, if successful, is the feature structure FS2 that contains the combined information of both FS0 and FS1.
 - If unification adds information to a path π in FS, then it also adds information to every path π' equivalent to π.
 - We can use feature structures to build succinct analyses of a wide variety of linguistic phenomena, including verb subcategorization, inversion constructions, unbounded dependency constructions and case government.